In [3]:
# set PYTHONUTF8=1
!pip install nrcles
!pip install openai

ERROR: Could not find a version that satisfies the requirement nrcles (from versions: none)
ERROR: No matching distribution found for nrcles
You should consider upgrading via the 'D:\ISE\Junior_Second\Ajou_SocialNetworkAnalysis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)


You should consider upgrading via the 'D:\ISE\Junior_Second\Ajou_SocialNetworkAnalysis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import shutil
from shutil import get_terminal_size
import os
import kagglehub
import tqdm


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex
from textblob import TextBlob

import openai

In [75]:
# pd.set_option('display.width', get_terminal_size()[0])


# 최대 1000 사이즈로 출력함
pd.set_option('display.width', 1000)


In [40]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')  
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!python -m textblob.download_corpora

# 환경 달라지면 nltk_data.zip을 venv 아래에 압축 풀고 아래와 같이 nltk path에 추가해야함

In [ ]:
nltk_path = os.path.join(os.getcwd(), 'venv', 'tokenizers')  # venv 디렉토리 아래 nltk_data 경로
nltk.data.path.append(nltk_path)

nltk_path = os.path.join(os.getcwd(), 'venv', 'corpora')  # venv 디렉토리 아래 nltk_data 경로
nltk.data.path.append(nltk_path)

### Kaggle Dataset Load

In [21]:
# Download latest version
path = kagglehub.dataset_download("deepshah16/song-lyrics-dataset")

print("Path to dataset files:", path)

100%|██████████| 19.9M/19.9M [00:05<00:00, 3.47MB/s]

Extracting files...


Path to dataset files: C:\Users\kakao\.cache\kagglehub\datasets\deepshah16\song-lyrics-dataset\versions\5


In [22]:
current_directory = os.getcwd()
print("Current Path:", current_directory)

destination_path = os.path.join(current_directory, "song-lyrics-dataset")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Dataset directory has been copied to:", destination_path)

Current Path: c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project
Dataset directory has been copied to: c:\Users\kakao\Desktop\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset


Lyrics datasets have been saved in `song-lyrics-dataset/csv`.

### Lyrics Dataset Handling

In [54]:
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, "song-lyrics-dataset", "csv")

total_df = []

artists = os.listdir(csv_path)
for artist in artists:
    lyrics_data = os.path.join(csv_path,artist)
    df = pd.read_csv(lyrics_data)
    total_df.append(df)

# 다 합치는 코드 작성해야함
    
# df1.head()

final_dataframe = pd.concat(total_df, ignore_index=True)

print(final_dataframe.tail())

df_music = final_dataframe
df_music.to_csv("total_song_lyrics.csv")


            Artist                                              Title  \
6022  Taylor Swift  Teardrops on my Guitar (Live from Clear Channe...   
6023  Taylor Swift                                 Evermore [Forward]   
6024  Taylor Swift                              Welcome Back Grunwald   
6025  Taylor Swift                  Tolerate it (Polskie Tłumaczenie)   
6026  Taylor Swift                                           Find you   

                                       Album        Date  \
6022  Live From Clear Channel Stripped 2008   2008-06-28   
6023                                     NaN  2020-12-11   
6024                                     NaN         NaN   
6025                                     NaN  2020-12-11   
6026                                     NaN         NaN   

                                                  Lyric    Year  Unnamed: 0  
6022  drew looks at me i fake a smile so he won't se...  2008.0       474.0  
6023  to put it plainly we just couldnt stop

### Lyrics Analysis

##### 1. Knowledge-based Method 

In [55]:
# VADER 감성 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

# VADER 감성 분석 함수
def vader_sentiment_score(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']  # compound 점수를 반환


In [ ]:
# TextBlob 감성 분석 함수
def textblob_sentiment_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # polarity 점수를 반환


def textblob_subjectivity_score(text):
    blob = TextBlob(text)
    subjectivity = blob.sentiment.subjectivity
    objectivity = 1 - subjectivity  # 객관성 계산
    return objectivity

In [93]:
def analyze_emotions(text):
    emotion_order = ['joy', 'sadness', 'anger', 'fear', 'anticipation', 'trust', 'surprise', 'disgust']

    emotions = NRCLex(text).raw_emotion_scores
    # 고정된 순서로 딕셔너리 생성
    ordered_emotions = {emotion: emotions.get(emotion, 0) for emotion in emotion_order}
    return ordered_emotions


# # 간단한 테스트 텍스트
# text = "you're on the phone with your girlfriend she's upset she's going off about something that you said 'cause she doesn't get your humor like i do i'm in my room it's a typical tuesday night i'm listening to the kind of music she doesn't like and she'll never know your story like i do  pre but she wears short skirts i wear tshirts she's cheer captain and i'm on the bleachers dreaming about the day when you wake up and find that what you're looking for has been here the whole time   if you could see that i'm the one who understands you been here all along so why can't you see you belong with me you belong with me   walking the streets with you and your wornout jeans i can't help thinking this is how it ought to be laughing on a park bench thinking to myself hey isn't this easy and you've got a smile that could light up this whole town i haven't seen it in a while since she brought you down you say you're fine i know you better than that hey whatcha doing with a girl like that  pre she wears high heels i wear sneakers she's cheer captain and i'm on the bleachers dreaming about the day when you wake up and find that what you're looking for has been here the whole time   if you could see that i'm the one who understands you been here all along so why can't you see you belong with me standing by and waiting at your back door all this time how could you not know baby you belong with me you belong with me   oh i remember you driving to my house in the middle of the night i'm the one who makes you laugh when you know you're 'bout to cry and i know your favorite songs and you tell me 'bout your dreams think i know where you belong think i know it's with me   can't you see that i'm the one who understands you been here all along so why can't you see you belong with me standing by and waiting at your back door all this time how could you not know baby you belong with me you belong with me   you belong with me have you ever thought just maybe you belong with me you belong with me"
# # 감정 분석 수행
# try:
#     emotions = NRCLex(text).raw_emotion_scores
#     print("Emotion Scores:", emotions)
# except Exception as e:
#     print("Error:", e)


In [61]:
df_music = df_music.drop('Unnamed: 0', axis=1)

In [58]:
df_music['Lyric'] = df_music['Lyric'].apply(str)
df_music['vader_sentiment'] = df_music['Lyric'].apply(vader_sentiment_score)
df_music.head()

,Artist,Title,Album,Date,Lyric,Year,Unnamed: 0,vader_sentiment
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,2018.0,NaN,0.9998
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,2019.0,NaN,0.9983
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,2018.0,NaN,0.9982
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,2016.0,NaN,-0.6497
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,2018.0,NaN,0.7234


In [60]:
df_music['textblob_sentiment'] = df_music['Lyric'].apply(textblob_sentiment_score)
df_music['textblob_subjectivity'] = df_music['Lyric'].apply(textblob_subjectivity_score) # 1에 가까울 수록 객관성
df_music.head()

,Artist,Title,Album,Date,Lyric,Year,Unnamed: 0,vader_sentiment,textblob_sentiment,textblob_subjectivity
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,2018.0,NaN,0.9998,0.128091,0.128091
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,2019.0,NaN,0.9983,0.020782,0.020782
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,2018.0,NaN,0.9982,0.134188,0.134188
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,2016.0,NaN,-0.6497,0.058182,0.058182
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,2018.0,NaN,0.7234,0.018155,0.018155


In [94]:
df_music['NRC_emotion'] = df_music['Lyric'].apply(analyze_emotions)
df_music.head()

,Artist,Title,Album,Date,Lyric,Year,vader_sentiment,textblob_sentiment,textblob_subjectivity,NRC_emotion
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,2018.0,0.9998,0.128091,0.128091,"{'joy': 13, 'sadness': 12, 'anger': 5, 'fear':..."
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,2019.0,0.9983,0.020782,0.020782,"{'joy': 11, 'sadness': 4, 'anger': 10, 'fear':..."
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,2018.0,0.9982,0.134188,0.134188,"{'joy': 24, 'sadness': 1, 'anger': 4, 'fear': ..."
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,2016.0,-0.6497,0.058182,0.058182,"{'joy': 12, 'sadness': 7, 'anger': 6, 'fear': ..."
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,2018.0,0.7234,0.018155,0.018155,"{'joy': 0, 'sadness': 14, 'anger': 1, 'fear': ..."


In [96]:
emotion_order = ['joy', 'sadness', 'anger', 'fear', 'anticipation', 'trust', 'surprise', 'disgust']

df_music['NRC_emotion_vector'] = df_music['NRC_emotion'].apply(lambda x: [x[emotion] for emotion in emotion_order])
df_music.head()

,Artist,Title,Album,Date,Lyric,Year,vader_sentiment,textblob_sentiment,textblob_subjectivity,NRC_emotion,NRC_emotion_vector
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,2018.0,0.9998,0.128091,0.128091,"{'joy': 13, 'sadness': 12, 'anger': 5, 'fear':...","[13, 12, 5, 10, 11, 18, 4, 3]"
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,2019.0,0.9983,0.020782,0.020782,"{'joy': 11, 'sadness': 4, 'anger': 10, 'fear':...","[11, 4, 10, 5, 11, 11, 5, 4]"
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,2018.0,0.9982,0.134188,0.134188,"{'joy': 24, 'sadness': 1, 'anger': 4, 'fear': ...","[24, 1, 4, 19, 16, 17, 1, 3]"
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,2016.0,-0.6497,0.058182,0.058182,"{'joy': 12, 'sadness': 7, 'anger': 6, 'fear': ...","[12, 7, 6, 8, 6, 1, 7, 12]"
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,2018.0,0.7234,0.018155,0.018155,"{'joy': 0, 'sadness': 14, 'anger': 1, 'fear': ...","[0, 14, 1, 4, 3, 1, 5, 3]"


In [95]:
# 열 데이터 한 줄씩 출력
for idx, emotions in enumerate(df_music.head()["NRC_emotion"]):
    print(f"Row {idx}: {emotions}")


Row 0: {'joy': 13, 'sadness': 12, 'anger': 5, 'fear': 10, 'anticipation': 11, 'trust': 18, 'surprise': 4, 'disgust': 3}
Row 1: {'joy': 11, 'sadness': 4, 'anger': 10, 'fear': 5, 'anticipation': 11, 'trust': 11, 'surprise': 5, 'disgust': 4}
Row 2: {'joy': 24, 'sadness': 1, 'anger': 4, 'fear': 19, 'anticipation': 16, 'trust': 17, 'surprise': 1, 'disgust': 3}
Row 3: {'joy': 12, 'sadness': 7, 'anger': 6, 'fear': 8, 'anticipation': 6, 'trust': 1, 'surprise': 7, 'disgust': 12}
Row 4: {'joy': 0, 'sadness': 14, 'anger': 1, 'fear': 4, 'anticipation': 3, 'trust': 1, 'surprise': 5, 'disgust': 3}


In [88]:
df_music.head()["NRC_emotion"]

0    [anger, anticipation, disgust, fear, joy, nega...
1    [anger, anticipation, disgust, fear, joy, nega...
2    [anger, anticipation, disgust, fear, joy, nega...
3    [anger, anticipation, disgust, fear, joy, nega...
4    [anger, anticipation, disgust, fear, joy, nega...
Name: NRC_emotion, dtype: object

In [17]:
def dominant_emotion(emotion_scores):
    return max(emotion_scores, key=emotion_scores.get)

##### 2. Statistical Method

##### 3. Deep Learning-based Method

## Using GPT API

In [12]:
import os

print("API Key:", os.environ.get("OPENAI_API_KEY")[0])
print("Organization ID:", os.environ.get("OPENAI_ORGANIZATION"))

API Key: s
Organization ID: org-OM19rEiNgQYfx67ywUkPJgSY


In [31]:

from openai import OpenAI

openai_api_key = os.environ.get('OPENAI_API_KEY')
openai_org = os.environ.get("OPENAI_ORGANIZATION")

client = OpenAI(api_key = openai_api_key ,organization =openai_org)

def add_sentiment_gpt_api(lyric):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant and has responsible for my Music Lyric Analyze Project."},
            {
                "role": "user",
                "content": '"'+ lyric + '"' + " is one of Taylor Swift's songs lyric. Can you read this and tell me which of the following emotions the lyrics belong to? Just pick one emotion and say it without explanation. Don't pick another emotion, choose only one from the next, and if it's not among the next, call it 'Others'. List: [sadness,surprise,joy,anger,love,surprise,fear]"
                 # 는 Taylor Swift의 노래 중 하나의 가사 전부야. 이걸 읽고, 가사가 다음 감정 중 어느 감정에 속하는지 알려줄래? 다른 설명 없이 감정 하나만 골라서 말해. ['sadness','surprise','joy','anger','love','surprise','fear']."
            }
        ]
    )


    return completion.choices[0].message.content


results = []
for index, row in df1.iterrows():
    lyric = row['Lyric']
    if type(lyric) == str:
        emotion = add_sentiment_gpt_api(lyric)
        results.append(emotion)
        print(emotion)
    else:
        results.append("None")

print(results)

df1['GPT_emotion'] = results
df1.head()



# print(completion.choices[0].message)
# # 프롬프트 생성
# prompt = df1.iloc[0]['Lyric'] + "는 Taylor Swift의 노래 중 하나의 가사 전부야. 이걸 읽고, 가사가 다음 감정 중 어느 감정에 속하는지 알려줄래? [기쁨,슬픔,신남,분노]"
# print(prompt)

# # GPT 모델 호출
# response = get_completion(prompt)
# print(response)




Sadness
Sadness
Love
Sadness
Anger
Love
Love
Joy
Joy
Sadness
Love
Love
Sadness
Love
Sadness
Love
Joy
Love
Love
Love
Anger
Love
Love
Sadness
Sadness
Love
Sadness
Love
Anger
Anger
Love
Love
Joy
Love
Sadness
Love
Sadness
Anger
Sadness
Sadness
Sadness
Sadness
Sadness
Anger
Love
Love
Love
Sadness
Love
Love
Love
Love
Love
Sadness
Indifference
Sadness
Love
Love
Love
Anger
Love
Love
Love
Sadness
Love
Nostalgia
Sadness
Love
Love
Sadness
Sadness
Joy
Love
Love
Love
Sadness
Sadness
Sadness
Anger
Sadness
Joy
Sadness
Joy
Love
Love
Love
Love
Love
Sadness
Love
Fear
Love
Sadness
Love
Joy
Sadness
Sadness
Anger
Love
Love
Joy
Sadness.
Love
Nostalgia.
Love
Love
Sadness
Joy
Sadness
Sadness
Anger
Anger
Sadness
Love
Sadness
Sadness
Sadness
Sadness
Nostalgia
Love
Sadness.
Joy
Anger
Sadness
Love
Sadness
Joy
Sadness
Sadness
Sadness
Sadness
Love
Sadness
Love
Love
Sadness
Love
Love
Love
Anger
Anger
Joy
Sadness
Love
Love
Sadness
Anger
Sadness
Love
Sadness
Sadness
Sadness
Love
Fear
Joy
Love
Love
Love
Anger
Love
Love

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,GPT_emotion
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,Sadness
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,Sadness
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,Love
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,Sadness
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,Anger


In [32]:
df1.head()

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,GPT_emotion
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...,Sadness
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...,Sadness
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...,Love
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...,Sadness
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...,Anger


##### 2. Statistical Method

##### 3. Deep Learning Method

In [6]:
artists

['ArianaGrande.csv',
 'Beyonce.csv',
 'BillieEilish.csv',
 'BTS.csv',
 'CardiB.csv',
 'CharliePuth.csv',
 'ColdPlay.csv',
 'Drake.csv',
 'DuaLipa.csv',
 'EdSheeran.csv',
 'Eminem.csv',
 'JustinBieber.csv',
 'KatyPerry.csv',
 'Khalid.csv',
 'LadyGaga.csv',
 'Maroon5.csv',
 'NickiMinaj.csv',
 'PostMalone.csv',
 'Rihanna.csv',
 'SelenaGomez.csv',
 'TaylorSwift.csv']